## **Dataset and Model Files**

In [0]:
import os
os.mkdir('Seq2Seq')

In [0]:
from google.colab import files
files.upload() #upload kaggle.json

Mount google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Download and process TIMIT dataset

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download mfekadu/darpa-timit-acousticphonetic-continuous-speech
!unzip -q darpa-timit-acousticphonetic-continuous-speech.zip -d Timit_dataset

In [0]:
import shutil
import os
os.mkdir('Seq2Seq/TIMIT')
shutil.move('Timit_dataset/data/TRAIN','Seq2Seq/TIMIT')
shutil.move('Timit_dataset/data/TEST','Seq2Seq/TIMIT')
shutil.rmtree('Timit_dataset', ignore_errors=True)
os.remove('darpa-timit-acousticphonetic-continuous-speech.zip')

Move to deepspeech directory

In [0]:
import os
os.chdir('Seq2Seq')

Upload model files

In [0]:
from google.colab import files
files.upload()

Prepare TIMIT dataset

In [0]:
!python3 prepare_timit.py

## **Model Training**

In [0]:
import constants as c
from dataset import create_dataset
from visualization import plot_accuracy, plot_loss
from model import create_model, create_optimizer, create_loss_function

In [0]:
X_train, X_test, y_train, y_test, y_lag_train, y_lag_test = create_dataset()

In [0]:
full_model, encoder_model, decoder_model = create_model(X_train.shape[1], y_train.shape[1])

full_model.summary()

optimizer = create_optimizer()

loss = create_loss_function()

full_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

from tensorflow.keras.utils import plot_model
plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)
plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)
plot_model(full_model, to_file='full_model.png', show_shapes=True)

In [0]:
history = full_model.fit(
    [X_train, y_train],
    y_lag_train,
    batch_size=c.batch_size,
    epochs=20,
    initial_epoch=0
    )

In [0]:
plot_accuracy(history)
plot_loss(history)

In [0]:
full_model.save_weights("full_model.h5")
encoder_model.save_weights("encoder_model.h5")
decoder_model.save_weights("decoder_model.h5")

## **Model Predction**

In [0]:
full_model.load_weights("/content/drive/My Drive/Colab Notebooks/full_model.h5")
encoder_model.load_weights("/content/drive/My Drive/Colab Notebooks/encoder_model.h5")
decoder_model.load_weights("/content/drive/My Drive/Colab Notebooks/decoder_model.h5")